# SQuAD Parser

Converts SQuAD json files into pandas ready csv files

In [1]:
pretrained_model = "albert-base-v2"
squad_version = "1.1"

In [2]:
from pathlib import Path
import os, json, pandas as pd
from transformers import AutoTokenizer
import sentencepiece
import re

In [3]:
model_name = re.findall(r"(.+?)-",pretrained_model)[0]; print(model_name)
tok = AutoTokenizer.from_pretrained(pretrained_model)
data_path = Path(f"../../data/SQuAD/{squad_version}")

albert


In [21]:
def squad_parser(directory: Path, tok , data_set: str="train"):
    """
    convert squad train and dev jsons to dfs
    """
    ds_dir = data_set + f"-v{squad_version}.json"
    with open(directory/ds_dir) as f:
        file = json.load(f)
    ques, paras, answers, idxs, seq_len = [],[],[],[],[]
    for item in file["data"]:
        for paragraphs in item["paragraphs"]:
            context = paragraphs["context"]
            tok_context = tok.tokenize(context)
            for qas in paragraphs["qas"]:
                for answer in qas["answers"]:
                    start_idx = len(tok.tokenize(context[:answer["answer_start"]]))
                    end_idx = start_idx + len(tok.tokenize(answer["text"]))
                    ques.append(qas["question"])
                    paras.append(context)
                    answers.append(tok_context[start_idx:end_idx])
                    idxs.append([start_idx,end_idx])
                    seq_len.append(len(tok_context + tok.tokenize(qas["question"])))
    return pd.DataFrame({"question":ques,"paragraph" : paras, "answer":answers, "idxs":idxs, "seq_len":seq_len})

if not os.path.exists(data_path/f"train_{model_name}.csv"):
    train = squad_parser(data_path, tok, "train")
    val = squad_parser(data_path, tok,"dev")

    # export to csv
    train.to_csv(data_path/f"train_{model_name}.csv",index=False)
    val.to_csv(data_path/f"val_{model_name}.csv",index=False)